# Data Visualisation and Exploratory Data Analysis (EDA)

## Overview
This notebook provides comprehensive data visualisation and exploratory data analysis of the global energy consumption and CO₂ emissions dataset. We'll use multiple visualisation libraries (Matplotlib, Seaborn, and Plotly) to create static and interactive visualisations.

## Learning Objectives
- Master different visualisation techniques for various data types
- Understand when to use static vs interactive visualisations
- Identify patterns, trends, and outliers through visual exploration
- Communicate insights effectively through data visualisation

## Libraries Used
- **Matplotlib**: Static, publication-quality plots
- **Seaborn**: Statistical visualisations with attractive defaults
- **Plotly**: Interactive, web-based visualisations

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set visualization styles
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10